# ファイル操作

In [1]:
#include <filesystem>
#include <iostream>
#include <fstream>
#include <cstdlib>
#include <string>

using namespace std;
namespace fs = filesystem;

* カレントディレクトリをホームに変更

In [2]:
fs::current_path(getenv("HOME")); nullptr; // getenv : <cstdlib>

(nullptr_t) nullptr_t


## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [3]:
auto result = fs::create_directory("Empty");
cout << (result ? "成功" : "失敗") << endl; nullptr;

成功


(nullptr_t) nullptr_t


* Blankという空ファイルを作成

In [4]:
{
	ofstream io("Blank",ios_base::trunc);
	io.close(); // ofstream : <fstream>
}

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [5]:
fs::create_symlink("../Blank","Empty/Symlink"); nullptr;

(nullptr_t) nullptr_t


* `fs::create_symlink("../Blank","Empty/Symlink")`  
	ディレクトリのシンボリックリンクは `fs::create_symlink` では作成できない場合があり,そんな場合でも確実にシンボリックリンクを生成する方法。
* `fs::create_hard_link("../Blank","Empty/Symlink")`  
	ハードリンクを生成する。

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [6]:
{
	ofstream io("Untitled.md",ios_base::out|ios_base::trunc);
	io << "# Header 1";
	io.close();
}

* ファイルの書き込みを行う領域全体をブロックで囲むと便利
```C++
{
	ofstream io("Untitled.md",ios_base::out);
	io << "# Header 1" << endl;
}
```
こうすることで,ファイルストリーム io は,ブロック内でのみ有効な変数となり,ブロックの終端に達すると `io` が破棄されて,同時に `close()` が自動的に実行されるため,本来は手動で呼び出す必要がない。
※ `close()` はオブジェクト `io` のディストラクタになっている

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [7]:
fs::rename("Empty","Package"); nullptr;

(nullptr_t) nullptr_t


* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [8]:
fs::rename("Package/Symlink","Package/Alias"); nullptr;

(nullptr_t) nullptr_t


* Untitled.mdを移動して,名称変更

In [9]:
fs::rename("Untitled.md","Package/Headers.md"); nullptr;

(nullptr_t) nullptr_t


## 追記
* Markdownファイルに追記

In [10]:
{
	ofstream io("Package/Headers.md",ios_base::out|ios_base::app);
	io << endl << "## Header 2" << endl << "### Header 3";
	io.close();
}

`ifstream`, `ofstream` のオプション
複数指定する場合は `|` で挟む (論理和ビット演算)

| | |
|:-|:-|
| `ios_base::in` | 読込可能にする |
| `ios_base::out` | 書込可能にする |
| `ios_base::app` | 追加書込 (書込前にポジションを終端に移動) |
| `ios_base::trunc` | ファイルを新規作成する (既に存在していれば内容が消去される) |
| `ios_base::binary` | バイナリを扱う |

## 読込み
* Markdownファイルを読込み

In [11]:
{
	ifstream io("Package/Headers.md",ios_base::in);
	string readText;
	while (io && getline(io,readText)) cout << readText << endl;
}

# Header 1
## Header 2
### Header 3


* 直接出力も可能
```C++
cout << fh << endl;
```


* `string` の変数にデータを格納するのであれば,矢印演算子で

```C++
io >> readText;
```

	としても大丈夫だが, `char` の場合は,サイズがわからないので,必ず次の方法で読み込むべきである。

```C++
io.getline(readText,10);
```

	読み込む文字数を制限できるので,オーバーすることがない。  
	コード中に使った方法はこれの応用であり,文字数を指定しない場合でも,1行毎に読み込まれて安定しやすい。

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [12]:
fs::create_directories("Package/Module/Submodule/Item");

(bool) true


- `fs::create_directory` の代わりに `fs::create_directories` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [13]:
for (const fs::directory_entry& e : fs::directory_iterator("Package")) std::cout << e.path() << std::endl;

"Package/Alias"
"Package/Module"
"Package/Headers.md"


* 再帰的にディレクトリを検索する場合は `recursive_directory_iterator` を使用する。
* `directory_entry` から `.path()` を使用したが,他にも `.exists()`, `.is_dorectory()`, `file_size()` などが使用できる

## ファイル/フォルダの複製
* BlankとModuleを複製

In [14]:
fs::copy_file("Blank","Package/Blank");
fs::copy("Package/Module","Package/Module Copy",fs::copy_options::recursive);

(void) nullptr


* 第3引数に上書きに関するオプションを指定できる (通常はファイルが存在しているとエラーになる)

| | |
|:-|:-|
| `skip_existing` | コピー先に既にファイルが存在していればコピーしない |
| `overwrite_existing` | コピー先に既にファイルが存在していても上書きコピーする |
| `update_existing` | コピー先に既にファイルが存在していたら,そのファイルがより古い場合に上書きコピーする |

* `fs::copy_file`,`fs::copy_symlink` とは異なり, `fs::copy` は何でもコピーできる
* `fs::copy_options::recursive` とすることで,ディレクトリ内の内容も再帰的にコピーされる

## ファイル/フォルダの削除

* ファイルBlankを削除

In [15]:
result = fs::remove("Blank");

(bool) true


* シンボリックリンクAliasを削除

In [16]:
result = fs::remove("Alias");

(bool) false


* フォルダModuleを削除

In [17]:
result = fs::remove_all("Package/Module");

(bool) true


- 空のフォルダは `fs::remove` で削除できる。空でない場合は削除できない。  
	`fs::remove_all` を使えば,空でなくても削除できる

## 権限を確認/変更

* 状態を確認する関数

In [18]:
auto check = [](string pt) -> void* {

	fs::path p = fs::path(pt);

	cout << (fs::exists(p) ? "存在しています" : "存在していません") << endl;

	fs::file_status s = fs::status(p);
	/*
		fs::status はファイル/フォルダの状態を示す
		シンボリックリンクについては...
		fs::status : リンク先のファイルの状態を示す
		fs::symlink_status : シンボリックリンク自体の状態を示す
	*/
	if (fs::is_symlink(p)) {
		// s.type()==fs::file_type::symlink と等価
		cout <<
		"シンボリックリンクです" << endl <<
		"リンク先: " << fs::read_symlink(p).native() << endl;
	}
	else if (fs::is_regular_file(p)) cout << "ファイルです" << endl; // s.type()==fs::file_type::regular と等価
	else if (fs::is_directory(p)) cout << "フォルダです" << endl; // s.type()==fs::file_type::directory と等価

	using pm = fs::perms;
	pm sp = s.permissions();
	if ((sp & pm::owner_read) == pm::owner_read) cout << "読込可能です" << endl;
	if ((sp & pm::owner_write) == pm::owner_write) cout << "書込可能です" << endl;
	if ((sp & pm::owner_exec) == pm::owner_exec) cout << "実行可能です" << endl;

	return nullptr;

};
nullptr; // ラムダ式の後に一度何かを出力しないとclingがクラッシュする

(nullptr_t) nullptr_t


* Markdownファイルの現在の状態を確認

In [19]:
check("Package/Headers.md");

存在しています
ファイルです
読込可能です
書込可能です


(void *) nullptr


* 読込/書込権限を剥奪

In [20]:
fs::permissions("Package/Headers.md",fs::perms(0044)); nullptr;

(nullptr_t) nullptr_t


* 状態を確認

In [21]:
check("Package/Headers.md");

存在しています
ファイルです


(void *) nullptr


* 読込/実行権限を付加

In [22]:
fs::permissions("Package/Headers.md",fs::perms(0544)); nullptr;

(nullptr_t) nullptr_t


* 状態を確認

In [23]:
check("Package/Headers.md");

存在しています
ファイルです
読込可能です
実行可能です


(void *) nullptr


- `fs::permissions` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	C++では8進数で表記するために,前に `0` を付加する

	e.g. `fs::permissions(somefile,fs::perms(0754))`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能